In [1]:
import ee

# # Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline


In [3]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


In [4]:
geoJSON = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            8.473892211914062,
                            49.98081240937428
                        ],
                        [
                            8.658599853515625,
                            49.98081240937428
                        ],
                        [
                            8.658599853515625,
                            50.06066538593667
                        ],
                        [
                            8.473892211914062,
                            50.06066538593667
                        ],
                        [
                            8.473892211914062,
                            49.98081240937428
                        ]
                    ]
                ]
            }
        }
    ]
}


In [5]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


In [6]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                  .first()
                  .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                  .filterBounds(aoi)
                  .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                  .first()
                  .clip(aoi))


In [7]:
ffa_db.bandNames().getInfo()


['VV', 'VH', 'angle']

In [8]:
url = ffa_db.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)


In [9]:
location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
rgb = ee.Image.rgb(ffa_db.select('VV'),
                   ffa_db.select('VH'),
                   ffa_db.select('VV').divide(ffa_db.select('VH')))

# Create the map object.
m = folium.Map(location=location, zoom_start=12)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(rgb, {'min': [-20, -20, 0], 'max': [0, 0, 2]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)
